In [208]:
import pandas as pd 
import numpy as np
import folium
import folium.plugins as plugins
from folium.plugins import HeatMapWithTime
from folium.plugins import HeatMap

In [191]:
# load data
mv_df = pd.read_csv('..\data\Motor_Vehicle_Collisions_-_Crashes.csv') 


In [193]:
mv_df.shape

(1604018, 29)

In [197]:
# Drop nulls
mv_df = mv_df.dropna(subset=['LATITUDE','LONGITUDE'])
mv_df.shape

(1407808, 29)

In [198]:
# filter on Brooklyn 
bk_df = mv_df[(mv_df.BOROUGH == 'BROOKLYN')]
bk_df.shape

(339271, 29)

In [221]:
peds_killed = bk_df['NUMBER OF PEDESTRIANS KILLED'] >= 1
pers_killed = bk_df['NUMBER OF PERSONS KILLED'] >= 1
pers_injured = mv_df['NUMBER OF PEDESTRIANS INJURED'] >= 1
pdk = bk_df.loc[peds_killed]
psk = bk_df.loc[pers_killed]
psi = bk_df.loc[pers_injured]
t= pdk.append(psk)
t.shape

(592, 29)

In [222]:
df_filtered.head()

ACCIDENT DATE ACCIDENT TIME   BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
1     04/27/2019          5:20  BROOKLYN    11235  40.592213 -73.932556   
6     04/02/2019         11:00  BROOKLYN    11221  40.686390 -73.942470   
12    04/12/2019         14:00  BROOKLYN    11203  40.654102 -73.930660   
15    04/27/2019          8:40  BROOKLYN    11209  40.631930 -74.036440   
17    04/25/2019         18:32  BROOKLYN    11233  40.675686 -73.913780   

                        LOCATION                    ON STREET NAME  \
1   POINT (-73.932556 40.592213)                               NaN   
6     POINT (-73.94247 40.68639)                               NaN   
12   POINT (-73.93066 40.654102)                               NaN   
15    POINT (-74.03644 40.63193)  77 STREET                          
17   POINT (-73.91378 40.675686)  BOYLAND STREET                     

   CROSS STREET NAME                           OFF STREET NAME  ...  \
1                NaN  2501      KNAPP STREET                    ...   
6                NaN  404       MONROE STREET                   ...   
12               NaN  790       UTICA AVENUE                    ...   
15    NARROWS AVENUE                                       NaN  ...   
17    PACIFIC STREET                                       NaN  ...   

    CONTRIBUTING FACTOR VEHICLE 2  CONTRIBUTING FACTOR VEHICLE 3  \
1                     Unspecified                            NaN   
6                     Unspecified                            NaN   
12                            NaN                            NaN   
15                    Unspecified                    Unspecified   
17                    Unspecified                            NaN   

    CONTRIBUTING FACTOR VEHICLE 4  CONTRIBUTING FACTOR VEHICLE 5  \
1                             NaN                            NaN   
6                             NaN                            NaN   
12                            NaN                            NaN   
15                            NaN                            NaN   
17                            NaN                            NaN   

    COLLISION_ID                  VEHICLE TYPE CODE 1  \
1        4122270                                Sedan   
6        4107489                                Sedan   
12       4113421                            Box Truck   
15       4121436  Station Wagon/Sport Utility Vehicle   
17       4121932                                Sedan   

                    VEHICLE TYPE CODE 2                  VEHICLE TYPE CODE 3  \
1                                 Sedan                                  NaN   
6                                   NaN                                  NaN   
12                                  NaN                                  NaN   
15  Station Wagon/Sport Utility Vehicle  Station Wagon/Sport Utility Vehicle   
17                                Sedan                                  NaN   

   VEHICLE TYPE CODE 4 VEHICLE TYPE CODE 5  
1                  NaN                 NaN  
6                  NaN                 NaN  
12                 NaN                 NaN  
15                 NaN                 NaN  
17                 NaN                 NaN  

[5 rows x 29 columns]

In [223]:
t.columns

Index(['ACCIDENT DATE', 'ACCIDENT TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [224]:
df_map = t.copy()

df_map['count'] = 1
df_map[['LATITUDE', 'LONGITUDE', 'count']] \
    .groupby(['LATITUDE', 'LONGITUDE']) \
    .sum() \
    .sort_values('count', ascending=False) \
    .head(10)

df_map.shape

(592, 30)

In [230]:
def generateBaseMap(default_location=[40.672765, -73.961033], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()
m = HeatMap(
        data = df_map[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE','LONGITUDE']).sum().reset_index().values.tolist() \
        , radius=7 \
        , max_zoom=5
    ).add_to(base_map)
m.save('../img/heatmapmv.html')
display(base_map)